In [440]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
import statsmodels
from sklearn import ensemble

from statsmodels.formula.api import ols
from statsmodels.graphics.api import abline_plot, interaction_plot
from statsmodels.stats.anova import anova_lm

sns.reset_defaults()
sns.set_style("whitegrid")
warnings.filterwarnings("ignore")

In [441]:
# for each node in a network, find the degree of its neighbors and take the average
# stores the degree and average neighbor degree in a dataframe
def get_avg_nn_degree(network):
    degrees = network.degree
    degree = []
    k_nearest_neighbors = []
    for i in network.nodes:
        k_i = degrees[i]
        if k_i > 0:
            network.nodes[i]['degree'] = k_i
            k_nn = sum([degrees[j] for j in network.neighbors(i)]) / k_i
            network.nodes[i]['avg_knn'] = k_nn
            degree.append(k_i)
            k_nearest_neighbors.append(k_nn)
        else:
            network.nodes[i]['degree'] = 0
            network.nodes[i]['avg_knn'] = 0
            degree.append(0)
            k_nearest_neighbors.append(0)
    return pd.DataFrame({'degree': degree, 'average_nearest_neighbor_degree': k_nearest_neighbors})

In [442]:
# for each network in the ensemble dataframe, gets the average nearest neighbor
# degree and puts it all in a dataframe
def get_avg_nn_degree_null(ensemble):
    network_nos = ensemble['network_no'].unique()
    null_nn = pd.DataFrame(columns=['degree', 'average_nearest_neighbor_degree'])
    for i in network_nos:
        null_edge_list = ensemble.loc[ensemble['network_no'] == i]
        null_network = nx.from_pandas_edgelist(null_edge_list, source='source', target='target')
        new_rows = get_avg_nn_degree(null_network)
        null_nn = pd.concat([null_nn, new_rows])
    return null_nn


### Finding line of best fit and $\mu$ for $k_{nn}(k)$:
For a non-neutral network, the line that best fits $k_{nn}(k)$ is given by: :
\begin{align}
k_{nn}(k) = ak^{\mu}
\end{align}
We need to find the values $a, \mu$ that best fit the data. Taking the logarithm of both sides gives us:
\begin{align}
log(k_{nn}(k)) &= log(ak^{\mu}) \\
&= log(a) + \mu log(k)
\end{align}
let:
- $K_{NN}(K) = log(k_{nn}(k))$
- $K = log(k)$
- $A = log(a)$

Substituting in these variables gives us a linear equation of the form:
\begin{align}
K_{nn}(K) = A + \mu K
\end{align}
We can use linear regression to find the values for $\mu$ and $A$, and find $a$ by using $a=10^A$.


In [443]:
def find_line(nn_df):
    # transform k and k_nn
    log_k = np.log10(nn_df['degree'].values)
    log_knn = np.log10(nn_df['average_nearest_neighbor_degree'].values)
    # add constant term for intercept (A)
    log_k = sm.add_constant(log_k)
    # Fit the model
    model = sm.OLS(log_knn, log_k).fit()

    return model

In [444]:
# Draws the k nearest neighbours plot
# def draw_knn(network, dp, rgg, genre, model):
def draw_knn(network, genre, model, ax, palette, l, model_dp):
    # Extract model parameters
    A, mu = model.params
    a = 10**A

    A_dp, mu_dp = model_dp.params
    a_dp = 10**A_dp

    # Get coordinates for line of best fit
    line_x = [k for k in network['degree'].unique()]
    line_y = [a * (x**mu) for x in line_x]

    # Group the network nearest neighbours by degree to make the scatterplot less cluttered
    network_avgs = network.groupby('degree')['average_nearest_neighbor_degree'].mean()
    max_degree = max(network_avgs.index)

    x_null = [k for k in network['degree'].unique()]
    y_null = [a_dp * (x**mu_dp) for x in x_null]

    # fig, ax = plt.subplots(dpi=300)
    ax.set(xscale='log',
           yscale='log',
           # xlim=(0,max_degree + 5),
           # ylim=(0,120),
           # xlabel='$ k $',
           ylabel = '$ k_{nn}(k) $',
           title=genre)
           # title=f'Average nearest neighbor degree for {genre} genre')

    sns.lineplot(x=line_x,
                 y=line_y,
                 ax=ax,
                 label = fr'$ {a:.2f}k^{{{mu:.2f}}} $',
                 alpha=.8,
                 color = palette[l])

    sns.lineplot(x = x_null,
                 y = y_null,
                 ax = ax,
                 label = fr'null model: $ {a_dp: .2f}k^{{{mu_dp:.2f}}} $',
                 color='grey')


    sns.scatterplot(x=network_avgs.index,
                              y=network_avgs.values,
                              ax=ax,
                              linewidth=.2,
                              s = 15,
                              alpha = 1,
                              color= palette[9 - l])

    # Draw nearest neighbor degree lines for null model ensembles
    # sns.lineplot(data=dp, x='degree', y='average_nearest_neighbor_degree', ax=ax, err_style='band', errorbar='sd', label = 'degree preserving model')
    # sns.lineplot(data=rgg, x='degree', y='average_nearest_neighbor_degree', ax=ax, err_style='band', errorbar='sd', label = 'random geometric model')
    # Draw line of best fit for network k_nn


    ax.legend(prop={'size': 6})
    # fig.savefig(f'./plots/k_nearest_neighbors/{genre}_knn.png')

    # plt.show()

In [445]:
def get_degree_probabilities_and_plot(network: nx.Graph, genre):
    max_degree = max(k for node, k in network.degree())

    degree_probability_matrix = get_degree_probabilities(network)

    source_degree = []
    target_degree = []
    p = []
    for j in range(max_degree):
        for k in range(max_degree):
            source_degree.append(j)
            target_degree.append(k)
            p.append(degree_probability_matrix[j, k])

    fig, ax = plt.subplots(dpi=300)
    heatmap_df = pd.DataFrame({'source degree $ j $': source_degree, 'target degree $ k $': target_degree, '$ e_{jk} $': p}) \
                   .pivot(index = 'source degree $ j $', columns = 'target degree $ k $', values = '$ e_{jk} $')

    ax.set(title=f'Degree probability matrix for edges in the {genre} genre network\n',
           xlabel='target degree $ j $',
           ylabel='source degree $ k $')

    sns.heatmap(heatmap_df,
                cbar_kws={'label': '$ e_{jk} $'},
                ax=ax,
                cmap='mako',
                xticklabels=10,
                yticklabels=10)


    fig.savefig(f'./plots/heatmap/{genre}_heatmap.png')
    # plt.show()

    return degree_probability_matrix

In [446]:
def get_degree_probabilities(network: nx.Graph):
    max_degree = max(k for node, k in network.degree())
    degree_count_matrix = np.zeros(shape=(max_degree, max_degree))
    degrees = network.degree

    # j = row
    # k = column
    # For every edge in the network, count the number of times a node with degree j
    # is connected to a node with degree k
    for e in network.edges:
        j = degrees[e[0]] - 1
        k = degrees[e[1]] - 1
        degree_count_matrix[j, k] += 1

    # divide by the total number of edges to get a probability distribution
    degree_probability_matrix = degree_count_matrix / network.number_of_edges()
    return degree_probability_matrix

In [447]:
# computes the assortativity coefficient r from the lecture on null models
def compute_r(prob_matrix):
    size = len(prob_matrix[0])
    q_j = np.sum(prob_matrix, axis=1)
    q_k = np.sum(prob_matrix, axis=0)
    differences = np.sum([(j*k)*(prob_matrix[j][k] - (q_j[j] * q_k[k])) for j in range(size) for k in range(size)])
    sigma = (np.sum([(k**2) * q_k[k] for k in range(size)])) - (np.sum([k * q_k[k] for k in range(size)])**2)
    return differences/sigma

In [448]:
def get_null_r(ensemble):
    network_nos = ensemble['network_no'].unique()
    null_rs = []
    for i in network_nos:
        null_edge_list = ensemble.loc[ensemble['network_no'] == i]
        null_network = nx.from_pandas_edgelist(null_edge_list, source='source', target='target')
        degree_probabilities = get_degree_probabilities(null_network)
        null_rs.append(compute_r(degree_probabilities))
    stats = np.array(statsmodels.stats.descriptivestats.Description(null_rs, stats = ['mean', 'ci'], alpha = 0.05).frame)
    return stats

In [449]:
def degree_correlation_metrics():
    genres = ['blues', 'classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
    metrics_df = pd.DataFrame(columns = ['genre', 'a', 'mu', 'mu MoE', 'r', 'null_a', 'null_mu', 'null_mu_MoE', 'null_r', 'null_r_moe'])
    fig, axs = plt.subplots(dpi=600, nrows=5, ncols=2, figsize=(8.5,11), sharex=True, sharey=True)
    l = 0
    palette = sns.color_palette('viridis', 10)
    k_nn_df = pd.DataFrame(columns=['genre', 'k', 'k_nn'])
    for genre in genres:
        print(f"calculating metrics for genre: {genre}")
        ensemble_dp = pd.read_csv(f"../null-model-ensembles/networks/degree_preserving/{genre}_dp.csv")
        # ensemble_rgg = pd.read_csv(f"../null-model-ensembles/networks/random_geometric/{genre}_rgg.csv")

        edge_list = pd.read_csv(f"../networks/{genre}_edge_list.csv").rename(columns={"Node1": "source", "Node2": "target"})
        network = nx.from_pandas_edgelist(edge_list, source='source', target='target')

        network_nn = get_avg_nn_degree(network).astype('float')
        dp_nn = get_avg_nn_degree_null(ensemble_dp).astype('float')
        # rgg_nn = get_avg_nn_degree_null(ensemble_rgg)

        new_rows = pd.DataFrame({'genre': [genre for i in range(network_nn.shape[0])],'k': network_nn['degree'].values, 'k_nn': network_nn['average_nearest_neighbor_degree'].values})
        k_nn_df = pd.concat([k_nn_df, new_rows])

        model = find_line(network_nn)
        A, mu = model.params
        a = 10**A

        model_dp = find_line(dp_nn)
        A_dp, mu_dp = model_dp.params
        a_dp = 10**A_dp

        # get confidence interval for mu
        cis = model.conf_int()
        mu_ci = cis[1]
        mu_moe = mu_ci[1] - mu

        null_cis = model_dp.conf_int()
        null_mu_ci = null_cis[1]
        mu_moe_null = null_mu_ci[1] - mu_dp

        i = int (l / 2)
        j = l % 2
        current_ax = axs[i][j]

        # draw graphs, and add degree correlation metrics to dataframe
        # draw_knn(network_nn, dp_nn, rgg_nn, genre, model)
        scatter = draw_knn(network_nn, genre, model, current_ax, palette, l, model_dp)
        degree_probability_matrix = get_degree_probabilities_and_plot(network, genre)

        r = compute_r(degree_probability_matrix)

        null_r_stats = get_null_r(ensemble_dp)
        r_null = null_r_stats[0,0]
        r_null_moe = (null_r_stats[1,0] - null_r_stats[2,0]) / 2

        metrics_df.loc[metrics_df.shape[0],:] = [genre, a, mu, mu_moe, r, a_dp, mu_dp, mu_moe_null, r_null, r_null_moe]

        l += 1

    # metrics_df.to_csv('degree_correlation_metrics.csv', index=False)

    axs[4][0].set(xlabel='degree $ k $')
    axs[4][1].set(xlabel='degree $ k $')
    fig.savefig(f'./plots/k_nearest_neighbors/all_genres_knn.png')

    return metrics_df, k_nn_df


In [450]:
metrics, k_nn_df = degree_correlation_metrics()

calculating metrics for genre: blues
calculating metrics for genre: classical
calculating metrics for genre: country
calculating metrics for genre: disco
calculating metrics for genre: hiphop
calculating metrics for genre: jazz
calculating metrics for genre: metal
calculating metrics for genre: pop
calculating metrics for genre: reggae
calculating metrics for genre: rock


seeing if genre has a significant impact on the value of $\mu$:

In [451]:
k_nn_df['log_k'] = np.log10(k_nn_df['k'])
k_nn_df['log_k_nn'] = np.log10(k_nn_df['k_nn'])

In [452]:
# Create linear regression model fitting k_nn against k, and has the genre as an interaction
k_nn_model = ols("log_k_nn ~ log_k*genre", data=k_nn_df).fit()
k_nn_model.summary()
#
# k_nn_model_mult = ols("log_k_nn ~ log_k*genre", data=k_nn_df).fit()
# print(k_nn_model_mult.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               log_k_nn   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     324.9
Date:                Mon, 07 Apr 2025   Prob (F-statistic):               0.00
Time:                        16:54:07   Log-Likelihood:                 131.27
No. Observations:                5153   AIC:                            -222.5
Df Residuals:                    5133   BIC:                            -91.60
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.8146      0.025     33.093      0.000       0.766       0.863
genre[T.classical]           0.1049      0.034      3.059      0.002       0.038       0.172
genre[T.country]             0.0511      0.035      1.481      0.139      -0.017       0.119
genre[T.disco]               0.0334      0.035      0.956      0.339      -0.035       0.102
genre[T.hiphop]             -0.0660      0.034     -1.917      0.055      -0.134       0.001
genre[T.jazz]                0.1684      0.036      4.661      0.000       0.098       0.239
genre[T.metal]               0.0873      0.036      2.447      0.014       0.017       0.157
genre[T.pop]                 0.1086      0.036      3.037      0.002       0.038       0.179
genre[T.reggae]             -0.0691      0.035     -1.977      0.048      -0.138      -0.001
genre[T.rock]                0.0017      0.033      0.052      0.958      -0.063       0.066
log_k                        0.5205      0.020     26.216      0.000       0.482       0.559
log_k:genre[T.classical]    -0.0727      0.028     -2.617      0.009      -0.127      -0.018
log_k:genre[T.country]      -0.0393      0.028     -1.404      0.160      -0.094       0.016
log_k:genre[T.disco]        -0.0255      0.028     -0.905      0.365      -0.081       0.030
log_k:genre[T.hiphop]        0.0566      0.028      2.029      0.043       0.002       0.111
log_k:genre[T.jazz]         -0.1283      0.029     -4.448      0.000      -0.185      -0.072
log_k:genre[T.metal]        -0.0712      0.029     -2.487      0.013      -0.127      -0.015
log_k:genre[T.pop]          -0.0827      0.029     -2.888      0.004      -0.139      -0.027
log_k:genre[T.reggae]        0.0468      0.028      1.650      0.099      -0.009       0.102
log_k:genre[T.rock]         -0.0096      0.027     -0.356      0.722      -0.063       0.043
==============================================================================
Omnibus:                      364.147   Durbin-Watson:                   1.640
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1794.649
Skew:                          -0.109   Prob(JB):                         0.00
Kurtosis:                       5.883   Cond. No.                         50.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [453]:
# create a linear regression model for all genres combined where the only independent variable is the degree
k_nn_model_no_interactions = ols("log_k_nn ~ log_k", data=k_nn_df).fit()
k_nn_model_no_interactions.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               log_k_nn   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     6001.
Date:                Mon, 07 Apr 2025   Prob (F-statistic):               0.00
Time:                        16:54:07   Log-Likelihood:                 86.833
No. Observations:                5153   AIC:                            -169.7
Df Residuals:                    5151   BIC:                            -156.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8530      0.008    109.060      0.000       0.838       0.868
log_k          0.4904      0.006     77.466      0.000       0.478       0.503
==============================================================================
Omnibus:                      345.899   Durbin-Watson:                   1.622
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1643.705
Skew:                          -0.088   Prob(JB):                         0.00
Kurtosis:                       5.761   Cond. No.                         4.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [454]:
# run a two-way ANOVA to see if the genre has a significant impact on the estimates of k_nn
anova_lm(k_nn_model_no_interactions,  k_nn_model)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,5151.0,291.708560,0.0,NaN,NaN,NaN
1,5133.0,286.720228,18.0,4.988332,4.961303,2.496797e-11
